# AdhocLoaderTool Tutorial

Our `AdhocLoaderTool` is a powerful agent tool that allows for "on-demand" data querying from any data source on LlamaHub.

This tool takes in a `BaseReader` data loader, and when called will 1) load data, 2) index data, and 3) query the data.

In this walkthrough, we show how to use the `AdhocLoaderTool` to convert our Wikipedia data loader into an accessible search tool for a LangChain agent.

In [1]:
from llama_index.tools.adhoc_loader_tool import AdhocLoaderTool
# from llama_hub.wikipedia.base import WikipediaReader
from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from pydantic import BaseModel

/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define Tool

We first define the `WikipediaReader`. Note that the `load_data` interface to `WikipediaReader` takes in a list of `pages`. By default, this queries the Wikipedia search endpoint which will autosuggest the relevant pages.

We then wrap it into our `AdhocLoaderTool`.

By default since we don't specify the `index_cls`, a simple vector store index is initialized.

In [2]:
reader = WikipediaReader()

In [3]:
class WikipediaLoaderSpec(BaseModel):
    """Wikipedia loader spec."""
    pages: List[str]
    query_str: str
    

tool = AdhocLoaderTool.from_defaults(
    reader,
    name="Wikipedia Tool",
    description="A tool for loading and querying articles from Wikipedia",
    fn_schema=WikipediaLoaderSpec
)

#### Testing

We can try running the tool by itself (or as a LangChain tool), just to showcase what the interface is like! 

Note that besides the arguments required for the data loader, the tool also takes in a `query_str` which will be
the query against the index.

In [5]:
# run tool by itself
tool(["Berlin"], query_str="What's the arts and culture scene in Berlin?")

['Berlin']
{}


"\nBerlin has a vibrant and diverse arts and culture scene. It is home to 44 theaters and stages, three major opera houses, and numerous art galleries. The cityscape of Berlin displays large quantities of urban street art, and the Berlin Wall has become one of the largest open-air canvasses in the world. Berlin also has a long history of gay culture, and is an important birthplace of the LGBT rights movement. There are many festivals and events throughout the year, such as the Berlin International Film Festival, the Karneval der Kulturen, the Berlin Festival, and the New Year's Eve celebrations. The city is also home to many museums, such as the Museum Island, the Gemäldegalerie, the Neue Nationalgalerie, the Pergamon Museum, the Bode Museum, the Hamburger Bahnhof, the German Museum of Technology, the Jewish Museum, the Museum für Naturkunde, the Kupferstichkabinett Berlin, the Museum Berggruen, and the Beate Uhse Erotic Museum."

In [9]:
# run tool as langchain structured tool
lc_tool = tool.to_langchain_structured_tool(verbose=True)
lc_tool.run(tool_input={"pages": ["Berlin"], "query_str": "What's the arts and culture scene in Berlin?"})

['Berlin']
{}


"\nBerlin has a vibrant and diverse arts and culture scene. It is home to 44 theaters and stages, three major opera houses, and numerous art galleries. The cityscape of Berlin displays large quantities of urban street art, and the Berlin Wall has become one of the largest open-air canvasses in the world. Berlin also has a long history of gay culture, and is an important birthplace of the LGBT rights movement. There are many festivals and events throughout the year, such as the Berlin International Film Festival, the Karneval der Kulturen, the Berlin Festival, and the New Year's Eve celebrations. The city is also home to many museums, such as the Museum Island, the Gemäldegalerie, the Neue Nationalgalerie, the Pergamon Museum, the Bode Museum, the Hamburger Bahnhof, the German Museum of Technology, the Jewish Museum, the Museum für Naturkunde, the Kupferstichkabinett Berlin, the Museum Berggruen, and the Beate Uhse Erotic Museum."

### Initialize LangChain Agent

For tutorial purposes, the agent just has access to one tool - the Wikipedia Reader

In [10]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

In [11]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True)

In [13]:
agent = initialize_agent(
    [lc_tool],
    llm=llm,
    agent="structured-chat-zero-shot-react-description",
    verbose=True
)

# Now let's run some queries! 

The AdhocLoaderTool allows the agent to simultaneously 1) load the data from Wikipedia, 2) query that data.

In [14]:
agent.run("Tell me about the arts and culture of Berlin")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Wikipedia Tool",
  "action_input": {
    "pages": ["Culture in Berlin"],
    "query_str": "Arts and culture in Berlin"
  }
}
```

['Culture in Berlin']
{}

Observation: 
Berlin is recognized as a world city of culture and creative industries. It is home to over 600 art galleries, 153 museums, and numerous cultural institutions. Berlin has established itself as a popular nightlife and entertainment center in Europe, with many young people, cultural entrepreneurs, and international artists settling in the city. The city is home to the Berlin Biennale for contemporary artists, held every June, and the Berlin International Film Festival, held annually in February. Berlin is also home to many international and regional television and radio stations, and the public broadcaster RBB has its headquarters in the city. The creative arts sector in Berlin comprises music, film, advertising, architecture, art, design, fashion, perfor


KeyboardInterrupt



In [17]:
agent.run("Tell me about the critical reception to The Departed")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Wikipedia Tool",
  "action_input": {
    "pages": ["The Departed"],
    "query_str": "critical reception"
  }
}
```

['The Departed']
{}

Observation: 
The critical reception of The Departed was overwhelmingly positive. On review aggregator Rotten Tomatoes, the film holds a 91% approval rating based on 284 reviews, with an average rating of 8.3/10. The website's critics consensus reads, "Featuring outstanding work from an excellent cast, The Departed is a thoroughly engrossing gangster drama with the gritty authenticity and soupy morality we have come to expect from Martin Scorsese." Metacritic, which uses a weighted average, assigned the film a score of 85 out of 100 based on 39 critics, indicating "universal acclaim". Audiences polled by CinemaScore gave the film an average grade of "A−" on an A+ to F scale. Entertainment Weekly ranked it on its end-of-the-decade "Best of" list, saying: "If they're lucky, directors ma

"The critical reception to The Departed was overwhelmingly positive, with an average rating of 8.3/10 on Rotten Tomatoes and a score of 85 out of 100 on Metacritic. It has been praised as an American epic tragedy and is considered one of Martin Scorsese's best films."